In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from collections import Counter

# read content of book
book_raw = pd.read_csv("../data/processed/Prinz_raw.csv")
titel = 'Der kleine Prinz'
author = 'Antoine de Saint-Exupéry'
names =['Ich', 'Prinz']

# give meta data about book

# copy raw content as a list to fist column second row   
book = book_raw.copy()

In [ ]:
## layout of book = [{'raw':[], 'meta': {}, 'word_list': [], 'word_bag': {}, '2-gram': [], '3-gram': []}] in seperate csv saved at the end of the notebook
# Note - per Hand entfert: , " da Probleme mit pd.read_csv
# Note - per Hand bereinigt: Inhaltsverzeichnis, erste letzte Seite, Kopf und Fußzeilen

In [ ]:
## tidy book

# convert everything to string
book = book.applymap(str)           # it should be string anyway

# save concat all rows in book to get a very long sentence without line breaks
sentence = ''.join(book['content'].tolist())

# remove: numbers
sentence = ''.join([i for i in sentence if not i.isdigit()])

# remove . : ! ? " - ;  etc pp w regex
sentence = re.sub('[^0-9a-zA-ZäüöÄÖÜß ]+', '', sentence)

# lower case everyting
sentence = sentence.lower()

#print(sentence)

In [ ]:
## book_list
book_lst = list(sentence.split())
alnumcheck = [book_lst[i].isalnum() for i in range(len(book_lst))]
book_list = pd.DataFrame(book_lst)
book_list = book_list.rename(columns={0: "list"})

In [ ]:
## book_bag
book_bag = {}

for e in book_lst:
    if e in book_bag:
        book_bag[e] += 1
    else:
        book_bag.update({e: 1})

book_bag = pd.DataFrame(book_bag.items(), columns=['word','freq'])

In [ ]:
## book_meta: titel, author, pages, number of all words, number of unique words, names (list), 
book_meta = {}

book_meta.update({'titel': titel})
book_meta.update({'author': author})
book_meta.update({'pages': [int(s) for s in re.findall(r'\b\d+\b', book.content[len(book['content'])-1])][-1]})
book_meta.update({'num_words': len(book_list['list'])})
book_meta.update({'num_uniq': book_list['list'].nunique()})
book_meta.update({'names': names})

book_meta = pd.DataFrame(book_meta.items(), columns=['prop','value'])

In [ ]:
## book_2gram
bigram_bag = Counter(zip(book_lst, book_lst[1:])).most_common()
bigram_list = [list(e) for e in zip(book_lst, book_lst[1:])]

book_2gramBag = pd.DataFrame(bigram_bag, columns=['bigram','freq'])
book_2gramList = pd.DataFrame(bigram_list, columns=['first','second'])

In [ ]:
## book_3gram
def find_ngrams(input_list, n):
  return zip(*(input_list[i:] for i in range(n)))

trigram_bag = Counter(find_ngrams(book_lst, 3)).most_common()
trigram_list = [list(e) for e in find_ngrams(book_lst, 3)]

book_3gramBag = pd.DataFrame(trigram_bag, columns=['trigram','freq'])
book_3gramList = pd.DataFrame(trigram_list, columns=['first','second', 'third'])


In [ ]:
## book_5gram
def find_ngrams(input_list, n):
  return zip(*(input_list[i:] for i in range(n)))

quadrigram_bag = Counter(find_ngrams(book_lst, 4)).most_common()
quadrigram_list = [list(e) for e in find_ngrams(book_lst, 4)]

book_4gramBag = pd.DataFrame(quadrigram_bag, columns=['quadrigram','freq'])
book_4gramList = pd.DataFrame(quadrigram_list, columns=['first','second', 'third', 'fourth'])

In [ ]:
## book_001gram
def find_ngrams(input_list, n):
  return zip(*(input_list[i:] for i in range(n)))

per = len(book_lst) // 100 
onepergram_bag = Counter(find_ngrams(book_lst, per)).most_common()
onepergram_list = [list(e) for e in find_ngrams(book_lst, per)]

# book_001gramBag = pd.DataFrame(onepergram_bag, columns=['onepergram','freq'])
book_001gramList = pd.DataFrame(onepergram_list)

In [ ]:
book_list.to_csv("../data/processed/Prinz_list.csv", index = False)
book_bag.to_csv("../data/processed/Prinz_bag.csv", index = False)
book_meta.to_csv("../data/processed/Prinz_meta.csv", index = False)
book_2gramList.to_csv("../data/processed/Prinz_2gramList.csv", index = False)
book_2gramBag.to_csv("../data/processed/Prinz_2gramBag.csv", index = False)
book_3gramList.to_csv("../data/processed/Prinz_3gramList.csv", index = False)
book_3gramBag.to_csv("../data/processed/Prinz_3gramBag.csv", index = False)
book_4gramList.to_csv("../data/processed/Prinz_4gramList.csv", index = False)
book_4gramBag.to_csv("../data/processed/Prinz_4gramBag.csv", index = False)
book_001gramList.to_csv("../data/processed/Prinz_001gramList.csv", index = False)
# book_001gramBag.to_csv("../data/processed/Prinz_001gramBag.csv", index = False)